In [2]:
# Import the dependencies.
import pandas as pd
import requests
# Import the API key.
from config import g_key

In [3]:
import gmaps

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cape Town,ZA,2022-05-24 23:05:01,-33.9258,18.4232,64.38,90,99,3.00
1,1,Port Alfred,ZA,2022-05-24 23:07:52,-33.5906,26.8910,60.19,92,85,1.01
2,2,Lebu,CL,2022-05-24 23:07:52,-37.6167,-73.6500,49.51,86,51,2.53
3,3,Matara,LK,2022-05-24 23:07:52,5.9485,80.5353,76.17,91,100,6.96
4,4,Quatre Cocos,MU,2022-05-24 23:07:53,-20.2078,57.7625,71.10,81,27,9.10


In [5]:
# get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
# configure gmaps to use the google api key
gmaps.configure(api_key=g_key)

In [7]:
# get the maximum temperatures
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp,0))

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig_gmap = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating = False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig_gmap.add_layer(heat_layer)
# Call the figure to plot the data.
fig_gmap

Figure(layout=FigureLayout(height='420px'))

In [9]:
# heatmap of percent humidity
locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]
fig_gmaps = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig_gmaps.add_layer(heat_layer)
fig_gmaps

Figure(layout=FigureLayout(height='420px'))

In [14]:
# ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [16]:
# filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Matara,LK,2022-05-24 23:07:52,5.9485,80.5353,76.17,91,100,6.96
5,5,Hithadhoo,MV,2022-05-24 23:07:53,-0.6000,73.0833,82.85,69,92,2.35
15,15,Mandawar,IN,2022-05-24 23:07:56,29.5000,78.1333,75.47,53,7,4.68
16,16,Mahuva,IN,2022-05-24 23:07:56,21.0914,71.7622,82.69,71,0,12.46
21,21,Bengkulu,ID,2022-05-24 23:07:58,-3.8004,102.2655,75.22,92,100,5.97


In [17]:
preferred_cities_df.count()

City_ID       151
City          151
Country       151
Date          151
Lat           151
Lng           151
Max Temp      151
Humidity      151
Cloudiness    151
Wind Speed    151
dtype: int64

In [18]:
# create dataframe called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Matara,LK,76.17,5.9485,80.5353,
5,Hithadhoo,MV,82.85,-0.6000,73.0833,
15,Mandawar,IN,75.47,29.5000,78.1333,
16,Mahuva,IN,82.69,21.0914,71.7622,
21,Bengkulu,ID,75.22,-3.8004,102.2655,
23,Rincon,US,81.91,32.2960,-81.2354,
24,Georgetown,MY,82.33,5.4112,100.3354,
36,Sinnamary,GF,78.46,5.3833,-52.9500,
45,Biloxi,US,84.20,30.3960,-88.8853,
49,Pemba,MZ,76.57,-12.9740,40.5178,


In [19]:
# set paramters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [ ]:
# iterate through the dataframe
for index, row in hotel_df.iterrows():
    # get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    # add the latitude and longitude to laction key for the params dictionary
    params["location"] = f"{lat},{lng}"
    # use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and get the json data from the search
    hotels = requests.get(base_url, params=params).json()
    # grab the first hotel from the results and store the name
try:  
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [21]:
# add a heatmap of temperature for the vacaiton spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
            max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# call the giure to plat the data
fig

NameError: name 'hotel_info' is not defined

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd> F</dd>
</dl>
"""
# store the dataframe row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]